In [1]:
!pip install pyowm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.7 MB/s eta 0:00:00


In [2]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.1-py3-none-any.whl size=215662 sha256=e56121d90a6c01aca12d7b77011f8a477c7dcacde18814bd3b0a2e4d1379ae5a
  Stored in directory: /root/.cache/pip/wheels/81/0e/bd/a68932e0d60008ec3eb11ae056e6d527af5ec9749922cd5a08
Successfully built pyTelegramBotAPI


In [4]:
import telebot
from pyowm import *

In [5]:
# owm - создаем подключение по Ключу к сервиску openweather
owm = OWM('bd5e378503939ddaee76f12ad7a97608')

# bot - создаем подключение по токену к боту
bot = telebot.TeleBot('6615988695:AAF9qkxs3RMqROx68pA5Hi_Bpo2Dqyqiwkc')

# mgr - создаем обьект для выполнения и отправки запросов к openweather
mgr = owm.weather_manager()

stroka = ''
@bot.message_handler(content_types=['text'])
# Функция, которая срабатывает при старте бота
def start(message):
    if message.text == '/help':
        bot.send_message(message.from_user.id, "Введите город и страну в формате - San Francisco, US");
        bot.register_next_step_handler(message, check); #следующий шаг – функция get_name
    else:
        bot.send_message(message.from_user.id, 'Напиши /help');

# Функция для проверки введенных данных и созданию системной кнопки, чтобы продолжитьи и вывести прогноз погоды
def check(message):
    global stroka;
    stroka = message.text;
    bot.send_message(message.from_user.id, "Подтвердите свои действия.")
    keyboard = telebot.types.InlineKeyboardMarkup(); #наша клавиатура
    key_yes = telebot.types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard.add(key_yes); #добавляем кнопку в клавиатуру
    question = 'Вы ввели ' + stroka;
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

# Обработчик нажатия, если человек подтвердил и нажал на кнопку ДА - сработает то, что внутри функции

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "yes":
        try:
            observation = mgr.weather_at_place(stroka)
            weather = observation.weather
            bot.send_message(call.message.chat.id, f"Today: {weather.detailed_status}")
            temp = weather.temperature('celsius')
            bot.send_message(call.message.chat.id, f"Temperature is: {temp['temp']}")
            bot.send_message(call.message.chat.id, f"Temperature max is: {temp['temp_max']}")
            bot.send_message(call.message.chat.id, f"Temperature min is: {temp['temp_min']}")
            bot.send_message(call.message.chat.id, f"Feels like: {temp['feels_like']}")
            wind = weather.wind()
            bot.send_message(call.message.chat.id, f"Speed of wind: {wind['speed']}")
            bot.send_message(call.message.chat.id, f"Gust: {wind['gust']}")
        except TypeError as e:
            print(e)

In [ ]:
# Штука, которая зацикливает работу бота и позвоялет обрабатывать много запросов
bot.polling(none_stop=True, interval=0)